## Description
In this part of the project, we will use supervised learning algorithms, namely, Decision Trees/Random Forests, SVM, and Logistic Regression algorithms to predict disease type given gene expression data.

## Import Packages

In [44]:
import pandas as pd
import numpy as np
import math
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.tree import plot_tree
# from sklearn.tree.export import export_text
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [26]:
#Settings
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

## Load in Data (Run this cell only once - large file)

In [91]:
data = pd.read_csv("combinedFPKM.csv")
data

,5ce879cb-fe9c-5cb9-9b2f-54779d6ad6c4,Cancer,ENSG00000000003.13,ENSG00000000005.5,ENSG00000000419.11,ENSG00000000457.12,ENSG00000000460.15,ENSG00000000938.11,ENSG00000000971.14,ENSG00000001036.12,ENSG00000001084.9,ENSG00000001167.13,ENSG00000001460.16,ENSG00000001461.15,ENSG00000001497.15,ENSG00000001561.6,ENSG00000001617.10,ENSG00000001626.13,ENSG00000001629.8,ENSG00000001630.14,ENSG00000001631.13,ENSG00000002016.15,ENSG00000002079.11,ENSG00000002330.12,ENSG00000002549.11,ENSG00000002586.16,ENSG00000002587.8,ENSG00000002726.18,ENSG00000002745.11,ENSG00000002746.13,ENSG00000002822.14,ENSG00000002834.16,ENSG00000002919.13,ENSG00000002933.6,ENSG00000003056.6,ENSG00000003096.12,ENSG00000003137.7,ENSG00000003147.16,ENSG00000003249.12,ENSG00000003393.13,ENSG00000003400.13,ENSG00000003402.18,ENSG00000003436.13,ENSG00000003509.14,ENSG00000003756.15,ENSG00000003987.12,ENSG00000003989.15,ENSG00000004059.9,ENSG00000004139.12,ENSG00000004142.10,...,ENSGR0000124333.13,ENSGR0000124334.15,ENSGR0000167393.15,ENSGR0000168939.9,ENSGR0000169084.11,ENSGR0000169093.13,ENSGR0000169100.11,ENSGR0000178605.11,ENSGR0000182162.8,ENSGR0000182378.11,ENSGR0000182484.13,ENSGR0000185203.10,ENSGR0000185291.9,ENSGR0000185960.11,ENSGR0000196433.10,ENSGR0000197976.9,ENSGR0000198223.13,ENSGR0000205755.9,ENSGR0000214717.8,ENSGR0000223274.4,ENSGR0000223484.5,ENSGR0000223511.4,ENSGR0000223571.4,ENSGR0000223773.5,ENSGR0000225661.5,ENSGR0000226179.4,ENSGR0000227159.6,ENSGR0000228410.4,ENSGR0000228572.5,ENSGR0000229232.4,ENSGR0000230542.4,ENSGR0000234622.4,ENSGR0000234958.4,ENSGR0000236017.6,ENSGR0000236871.5,ENSGR0000237040.4,ENSGR0000237531.4,ENSGR0000237801.4,ENSGR0000263835.4,ENSGR0000263980.4,ENSGR0000264510.4,ENSGR0000264819.4,ENSGR0000265658.4,ENSGR0000270726.4,ENSGR0000275287.3,ENSGR0000276543.3,ENSGR0000277120.3,ENSGR0000280767.1,ENSGR0000281849.1,id
0,NaN,Brease,9.342228,0.828756,19.959239,3.713461,1.577598,1.557466,3.895561,25.113650,8.062482,19.397789,0.745033,5.288839,16.220930,5.519227,37.908338,0.004312,23.510775,2.397773,8.385777,1.551339,0.030864,6.536317,26.057553,69.563461,0.361272,0.470571,0.018769,0.023667,2.868146,59.892201,6.735070,1.713565,26.245626,0.499608,0.456454,11.350259,5.723320,3.616736,0.667002,2.787415,0.825559,2.139119,8.430035,0.144091,32.211317,79.499496,1.969681,63.630315,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,NaN,Brease,19.658110,0.000000,34.302484,4.674391,3.941016,1.505171,2.325878,47.765837,2.832710,13.435799,1.060891,2.509758,15.209583,12.459083,24.168524,0.077162,17.797365,4.039856,8.870517,3.164233,0.022513,38.594390,32.461384,73.974563,1.686561,0.209221,0.052573,0.006138,7.263930,213.316385,11.291551,2.882817,20.562074,0.053194,0.290579,23.350756,23.366987,2.859008,0.819826,3.557586,0.348815,3.224407,8.113482,0.115459,0.271096,173.976522,0.856925,60.986580,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,NaN,Brease,1.028690,0.098894,129.116200,3.580859,1.491591,0.316237,1.213933,16.241104,4.633766,14.292499,1.556456,8.963247,10.047671,14.436147,57.131994,0.003739,10.763451,2.438696,8.974658,2.409180,0.030108,26.185322,22.156006,24.105826,0.776011,0.008327,0.004883,0.092350,3.957344,80.251992,8.963226,4.407232,25.157986,0.269051,1.755692,11.995929,24.007427,4.043333,0.389769,1.951982,0.440386,4.949691,13.137692,2.601430,75.858994,68.104071,1.572601,52.344739,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,NaN,Brease,14.286209,0.000000,26.467142,9.394148,2.610714,1.062695,4.011221,13.538960,8.880911,20.320788,2.266629,7.531598,9.140765,12.365308,17.293

## Preprocessing

In [89]:
data = data.iloc[0:6728,1:60485]
#data = data.rename(columns = {'Cancer': 'Cancer Type'})
data

,ENSG00000000460.15,ENSG00000000938.11,ENSG00000000971.14,ENSG00000001036.12,ENSG00000001084.9,ENSG00000001167.13,ENSG00000001460.16,ENSG00000001461.15,ENSG00000001497.15,ENSG00000001561.6,ENSG00000001617.10,ENSG00000001626.13,ENSG00000001629.8,ENSG00000001630.14,ENSG00000001631.13,ENSG00000002016.15,ENSG00000002079.11,ENSG00000002330.12,ENSG00000002549.11,ENSG00000002586.16,ENSG00000002587.8,ENSG00000002726.18,ENSG00000002745.11,ENSG00000002746.13,ENSG00000002822.14,ENSG00000002834.16,ENSG00000002919.13,ENSG00000002933.6,ENSG00000003056.6,ENSG00000003096.12,ENSG00000003137.7,ENSG00000003147.16,ENSG00000003249.12,ENSG00000003393.13,ENSG00000003400.13,ENSG00000003402.18,ENSG00000003436.13,ENSG00000003509.14,ENSG00000003756.15,ENSG00000003987.12,ENSG00000003989.15,ENSG00000004059.9,ENSG00000004139.12,ENSG00000004142.10,ENSG00000004399.11,ENSG00000004455.15,ENSG00000004468.11,ENSG00000004478.7,ENSG00000004487.14,ENSG00000004534.13,...,ENSGR0000002586.16,ENSGR0000124333.13,ENSGR0000124334.15,ENSGR0000167393.15,ENSGR0000168939.9,ENSGR0000169084.11,ENSGR0000169093.13,ENSGR0000169100.11,ENSGR0000178605.11,ENSGR0000182162.8,ENSGR0000182378.11,ENSGR0000182484.13,ENSGR0000185203.10,ENSGR0000185291.9,ENSGR0000185960.11,ENSGR0000196433.10,ENSGR0000197976.9,ENSGR0000198223.13,ENSGR0000205755.9,ENSGR0000214717.8,ENSGR0000223274.4,ENSGR0000223484.5,ENSGR0000223511.4,ENSGR0000223571.4,ENSGR0000223773.5,ENSGR0000225661.5,ENSGR0000226179.4,ENSGR0000227159.6,ENSGR0000228410.4,ENSGR0000228572.5,ENSGR0000229232.4,ENSGR0000230542.4,ENSGR0000234622.4,ENSGR0000234958.4,ENSGR0000236017.6,ENSGR0000236871.5,ENSGR0000237040.4,ENSGR0000237531.4,ENSGR0000237801.4,ENSGR0000263835.4,ENSGR0000263980.4,ENSGR0000264510.4,ENSGR0000264819.4,ENSGR0000265658.4,ENSGR0000270726.4,ENSGR0000275287.3,ENSGR0000276543.3,ENSGR0000277120.3,ENSGR0000280767.1,ENSGR0000281849.1
1,3.941016,1.505171,2.325878,47.765837,2.832710,13.435799,1.060891,2.509758,15.209583,12.459083,24.168524,0.077162,17.797365,4.039856,8.870517,3.164233,0.022513,38.594390,32.461384,73.974563,1.686561,0.209221,0.052573,0.006138,7.263930,213.316385,11.291551,2.882817,20.562074,0.053194,0.290579,23.350756,23.366987,2.859008,0.819826,3.557586,0.348815,3.224407,8.113482,0.115459,0.271096,173.976522,0.856925,60.986580,7.605533,23.145238,0.541877,120.185496,19.057807,11.790216,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.491591,0.316237,1.213933,16.241104,4.633766,14.292499,1.556456,8.963247,10.047671,14.436147,57.131994,0.003739,10.763451,2.438696,8.974658,2.409180,0.030108,26.185322,22.156006,24.105826,0.776011,0.008327,0.004883,0.092350,3.957344,80.251992,8.963226,4.407232,25.157986,0.269051,1.755692,11.995929,24.007427,4.043333,0.389769,1.951982,0.440386,4.949691,13.137692,2.601430,75.858994,68.104071,1.572601,52.344739,6.889993,18.193244,0.009291,49.373162,32.787039,13.451385,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.610714,1.062695,4.011221,13.538960,8.880911,20.320788,2.266629,7.531598,9.140765,12.365308,17.293903,0.082684,10.475410,1.309518,6.511355,1.981199,0.043716,12.544067,35.716858,46.994581,0.496520,0.066966,0.032720,0.042787,2.762677,52.855634,7.634076,1.435337,22.611148,0.326990,0.599985,24.031937,5.037661,3.958076,0.508361,4.957747,0.586601,3.653093,11.871970,0.284932,0.348241,52.716755,0.770283,57.218317,6.832802,21.708800,0.065374,72.296143,21.932791,9.942257,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.860832,3.011771,7.098766,14.828362,8.059247,15.627057,1.870120,4.607655,11.168068,0.931820,7.327714,0.02709

In [88]:
data = data.iloc[0:6728,1:60485]
count = 0
# for idx,val, in enumerate(check):
#     #print(type(val))
#     if(type(val) == float):
#         #print(math.isnan(val))
#         if math.isnan(val):
#             count+=1
#     else:
#         print(val)
#         print(type(val))
        
#print(check.iloc[0,:])
print(data.columns.values)

['ENSG00000000457.12' 'ENSG00000000460.15' 'ENSG00000000938.11' ...
 'ENSGR0000277120.3' 'ENSGR0000280767.1' 'ENSGR0000281849.1']


In [ ]:
# Reading data & split into training and testing sets
X = data.drop(['Crime Score'], 1)
print(X)
y = data['Crime Score']
print(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=12345)

#Model Definition & Fitting
clf = DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

# Predict based on the given features for the test set
y_pred = clf.predict(X_test)
predictions_probability = clf.predict_proba(X_test)